In [1]:
import numpy as np

---

# Item XIX

Implement the Conjugate Gradient Method with symmetric preconditioner. Select a convenient problem of your choice, but not a trivial one and test it. Please describe the problem and implementation completely.

---

In [2]:
# This is a regular version of the Conjugate Gradient, but generalized
# so the preconditioner can be added as a function that multiplies a vector
# (by default this function keeps it as it is)
def conjugate_gradient(A,b,pinv=lambda x:x,x0=None,threshold=1e-12):
    n = A.shape[0]
    if x0 is None: x0 = np.zeros(n)
    #
    r0 = b - np.dot(A,x0) # Initial residual.
    z0 = pinv(r0) # Initial residual after using the preconditioner.
    p0 = z0 # First conjugate vector (P^-1 A) (basis)
    #
    xk = x0
    rk = r0
    pk = p0
    zk = z0
    #
    for k in range(n):
        ak = np.dot(rk,zk)/np.dot(pk,np.dot(A,pk))
        # ^ coefficient for pk on x that minimizes next residual
        xk1 = xk + ak*pk
        # ^ Update x, add the projection on the direction of the new conjugate vector
        rk1 = rk - ak*np.dot(A,pk)
        # ^ Update residual
        zk1 = pinv(rk1)
        # ^ Residual after using preconditioner
        divi = np.dot(zk,rk)
        if np.abs(divi)<threshold: break # Terminate if residual is too small
        bk = np.dot(zk1,rk1)/divi
        pk1 = zk1 + bk*pk
        # ^ Next conjugate vector
        #
        # Move forward
        xk = xk1
        rk = rk1
        pk = pk1
        zk = zk1
    return xk1

In [3]:
def random_positive_definite_diag_dominant_matrix(n):
    # We create a simetric matrix multiplying a matrix by itself
    # the values decrease fast while descending with the diagonal
    # (the idea is to make it ill-conditioned)
    a = np.random.random((n,n)) * (np.arange(1.0,n+1)**-4)
    A = np.dot(a,a.T)
    # We change the diagonal to make it diagonal dominant
    sumr = np.sum(A,axis=0)
    np.fill_diagonal(A,sumr)
    return A
# Just test code.
N = 5
A = random_positive_definite_diag_dominant_matrix(N)
b = np.random.random(N)
print(A)
print(b)
x1 = np.linalg.solve(A,b)
x2 = conjugate_gradient(A,b)
print(x1)
print(x2)

[[3.71413712 0.9620883  0.44807942 0.93563933 0.42363722]
 [0.9620883  3.78608239 0.45631339 0.95419135 0.43249863]
 [0.44807942 0.45631339 1.76162215 0.44376439 0.20093234]
 [0.93563933 0.95419135 0.44376439 3.68251236 0.42075958]
 [0.42363722 0.43249863 0.20093234 0.42075958 1.66880361]]
[0.62500652 0.06863122 0.42105896 0.62394209 0.93272038]
[ 0.09380268 -0.10832627  0.15998423  0.09497974  0.51996736]
[ 0.09380268 -0.10832627  0.15998423  0.09497974  0.51996736]


Let's consider the Jacobi preconditioner (the preconditioner is the diagonal of the matrix, so the inverse is simple), that is good for diagonal dominant matrices and also is simmetrical.

In [4]:
def jacobi_preconditioner_inv(A):
    P = np.diag(A)**-1
    return lambda x: P*x

In [5]:
NS = (10,100,1000,2000)

for nn in NS:
    AA = random_positive_definite_diag_dominant_matrix(nn)
    bb = np.random.random(nn)
    xx = conjugate_gradient(AA,bb)
    xx2 = conjugate_gradient(AA,bb,pinv=jacobi_preconditioner_inv(AA))
    # Check error:
    print("For n=%d:"%nn)
    err = np.mean(np.abs(np.dot(AA,xx)-bb))
    print("\tregular error: "+str(err))
    err2 = np.mean(np.abs(np.dot(AA,xx2)-bb))
    print("\tprecond error: "+str(err2))

For n=10:
	regular error: 1.1537243382875318e-12
	precond error: 7.099800403143064e-09
For n=100:
	regular error: 1.6401885656949777e-08
	precond error: 5.764102297550755e-10
For n=1000:
	regular error: 1.4263998411277735e-08
	precond error: 4.822434864631964e-13
For n=2000:
	regular error: 1.4045056901688947e-08
	precond error: 6.827028872780083e-14


We can see that it allows us to decrease the error when $n$ gets larger.